In [7]:
import numpy as np
import pandas as pd
import scipy.stats as st

In [8]:
nsim = 20000

In [9]:
def gen_data(n, sims):
    np.random.seed(1015033030)
    
    ids = []  # Generating simulation IDs
    for i in range(sims):
        ids.extend([i + 1] * n)

    df = pd.DataFrame()
    df['sim_id'] = ids

    # Creating confounders
    df['age'] = np.round(np.random.normal(65, 10, size=n * sims), 0)  
    df['crcl_log'] = np.random.normal(np.log(110), 0.18, size=n * sims) - 0.005 * df['age']
    df['crcl'] = np.exp(df['crcl_log'])
    df['diabetes'] = np.random.binomial(n=1, p=st.logistic.cdf(-6.73 + 0.03 * df['crcl_log'] + 0.02 * df['age'] +
                                                            0.0009 * df['age'] ** 2), size=n * sims)
    df['insulin'] = np.where(df['diabetes']==1,
                        np.random.binomial(n=1, p=st.logistic.cdf(-4.16 + 0.04 * df['crcl_log'] - 0.02 * df['age'] +
                                                            0.0009 * df['age'] ** 2), size=n * sims),
                             0)
    df['lvef'] = np.random.beta(11, 7, size=n * sims)*100 - 0.06 * df['age']
    df['smoking'] = np.random.binomial(n=1, p=.21, size=n * sims)
    df['pvd'] = np.random.binomial(n=1, p=st.logistic.cdf(-5.62 + 0.03 * df['smoking'] - 0.02 * df['age'] +
                                                            0.0009 * df['age'] ** 2), size=n * sims)
    df['copd'] = np.random.binomial(n=1, p=st.logistic.cdf(-2.71 + 0.03 * df['smoking'] + 0.01 * df['pvd']), size=n * sims)

    df['tvd_lmcad'] = st.betabinom.rvs(2, .4, .7, loc=0, size=n * sims, random_state=None) #nb 0=3vd only, 1=lmcad only, 2=both

    # One-hot encoding
    df['tvd'] = np.where(df['tvd_lmcad']==0, 1, 0)
    df['lmcad'] = np.where(df['tvd_lmcad']==1, 1, 0)
    df['both'] = np.where(df['tvd_lmcad']==2, 1, 0)

    ## Define and run a function that generates anatomical SYNTAX scores
    def sim_anat_syntax(tvd_lmcad):
        """simulate anatomic syntax scores from coronary disease type."""
        from zepid.sensitivity_analysis import trapezoidal

        dummy_mat=np.stack([tvd_lmcad==0, tvd_lmcad==1, tvd_lmcad==2], axis=1)

        tvd=trapezoidal(3,10,10,50, len(tvd_lmcad))
        lmcad=trapezoidal(4,20,20,50, len(tvd_lmcad))
        both=trapezoidal(7,10,30,60, len(tvd_lmcad))

        return(dummy_mat[:,0]*tvd + dummy_mat[:,1]*lmcad + dummy_mat[:,2]*both)

    df['syntax']= np.round(sim_anat_syntax(df['tvd_lmcad']))

    ## Treatment allocation mechanism (True Propensity Score)
    df['cabg_pr'] = st.logistic.cdf(-2.971
                               + 0.049 * (df['age'] - 30)
                               - 0.001 * (df['age'] - 30)**2
                               + 0.212 * df['crcl_log']
                               + 0.973 * np.where(df['crcl_log'] > np.log(100), 1, 0)
                               - 0.386 * df['copd']
                               # Treatment-assignment based on disease type
                               + 1.973 * df['lmcad']
                               + 2.973 * df['both']
                               )
    df['cabg'] = np.random.binomial(n=1, p=df['cabg_pr'], size=n*sims)

    ## Potential outcomes
    def syntax2020(age, crcl, lvef, copd, pvd, diabetes, insulin, smoking, tvd, lmcad, syntax, cabg):
        return 1-np.exp(-0.243 *np.exp(0.99 * (0.72*age/10 - 0.07 * np.where(crcl<90, crcl ,90)/10 -0.31 * np.where(lvef<50, lvef, 50)/10 + 0.48 * copd + 0.73 * pvd + 0.20 * diabetes 
                    + 0.46 * insulin + 0.66 * smoking)
                    - 0.4 * cabg * tvd - 0.08 * cabg * lmcad - 0.1 * (1 - cabg) * lmcad + .16 * (1-cabg) * (syntax - 29)/10 -2.80))
    
    df['prY1'] = df.apply(lambda row : syntax2020(age=row['age'], crcl=np.exp(row['crcl_log']), lvef=row['lvef'],
                                copd=row['copd'], pvd=row['pvd'], diabetes=row['diabetes'],
                                 insulin=row['insulin'], smoking=row['smoking'], tvd=row['tvd_lmcad']==0,
                                 lmcad=row['tvd_lmcad']==1, syntax=row['syntax'],
                                 cabg=1), axis = 1)

    df['Y1'] = np.random.binomial(n=1, p=df['prY1'], size=n*sims)

    df['prY0'] = df.apply(lambda row : syntax2020(age=row['age'], crcl=np.exp(row['crcl_log']), lvef=row['lvef'],
                                copd=row['copd'], pvd=row['pvd'], diabetes=row['diabetes'],
                                 insulin=row['insulin'], smoking=row['smoking'], tvd=row['tvd_lmcad']==0,
                                 lmcad=row['tvd_lmcad']==1, syntax=row['syntax'],
                                 cabg=0), axis = 1)

    df['Y0'] = np.random.binomial(n=1, p=df['prY0'], size=n*sims)

    df['Y'] = np.where(df['cabg'] == 1, df['Y1'], df['Y0'])  # causal consistency

    df['true_cate'] = df['prY1'] - df['prY0']

    df['pred_cate'] = df['true_cate']
    
    return df

In [10]:
large_df = gen_data(nsim, 1)

#### Define functions for the true ARE and true ASREs (three stochastic implementations as described below)

In [11]:
def compute_true_are(df):
    """Compute true ARE from the true ps and the true cate"""
    true_are = np.mean( ( (df.pred_cate<0) - df.cabg_pr)*df.true_cate)
    return true_are

In [12]:
def compute_true_r_asre(df, alpha):
    """Compute true uniform/random ASRE from the true ps and the true cate
    alpha is the uniform/random parameter (bounded between 0 and 1) """
    df_temp = df
    df_temp['p_x'] = np.repeat(alpha, len(df))
    true_r_asre = np.mean( df_temp.p_x * ( (df_temp.pred_cate<0) - df_temp.cabg_pr)*df_temp.true_cate)
    return true_r_asre

In [13]:
def syntax_iHR_ci(df, alpha=.05):
    """Get individualized HR and 1-alpha CI from SYNTAX 2020"""
    
    coef_mod2=np.array([0.99, -0.40, -0.08, -0.10, 0.16])
    VCOV=np.array(  [[3.17E-03, -1.58E-04, -2.68E-04,2.56E-04,-3.78E-04],
                     [-1.58E-04,1.53E-02,6.56E-03,6.48E-03,1.45E-04],
                     [-2.68E-04,6.56E-03,1.68E-02,6.47E-03,1.62E-04],
                     [2.56E-04,6.48E-03,6.47E-03,1.75E-02,-1.08E-03],
                     [-3.78E-04,1.45E-04,1.62E-04,-1.08E-03,2.84E-03]])
    
    X_cabg=np.stack([np.repeat(99,len(df)), 1* (df.tvd_lmcad==0), 1* (df.tvd_lmcad==1), 0 * np.array(df.tvd_lmcad==1), 0 * (df.syntax-29) / 10]).T # NB PIs don't contribute as it cancel out
    X_pci=np.stack([np.repeat(99,len(df)), 0* (df.tvd_lmcad==0), 0* (df.tvd_lmcad==1), 1 * np.array(df.tvd_lmcad==1), 1 * (df.syntax-29) / 10]).T # in the substraction (here set at 99)
    X_diff=X_cabg-X_pci
    log_iHR = np.matmul(X_diff, coef_mod2)
    df_temp = df.copy()
    df_temp['pred_hr'] = np.exp(log_iHR)
    
    log_iHR_se=X_diff.dot(VCOV).dot(X_diff.T).diagonal()
    
    temp = np.stack( [log_iHR, log_iHR-st.norm.ppf(1-alpha/2)*log_iHR_se, log_iHR+st.norm.ppf(1-alpha/2)*log_iHR_se])
    iHR_CI = np.exp(temp.T)
    df_temp['iHR']= iHR_CI[:,0]
    df_temp['iHR_lb']= iHR_CI[:,1]
    df_temp['iHR_ub']= iHR_CI[:,2]
    
    significant = ((df_temp['iHR_lb']<1) == (df_temp['iHR_ub']<1))
    
    df_temp['significance'] = significant
    
    return df_temp

def compute_true_ci_asre(df, alpha):
    """Compute true CI ASRE from the true ps and the true cate"""
    df_temp = syntax_iHR_ci(df, alpha)
    true_ci_asre = np.mean( df_temp.significance * ( (df_temp.pred_cate<0) - df_temp.cabg_pr)*df_temp.true_cate)
    return true_ci_asre

In [14]:
def legit(x):
    return .5 * np.log((x+1)/(1-x))

def compute_true_cb_asre(df, alpha):
    """Compute true cognitive biais ASRE from the true ps and the true cate,
    alpha is the cognitive biais parameter (bounded between 0 and 1) """
    df_temp = df
    df_temp['p_x'] = (1 - np.abs( (df_temp['pred_cate']<0) - df_temp['cabg_pr']) ) ** legit(alpha)
    true_cb_asre = np.mean( df_temp.p_x * ( (df_temp.pred_cate<0) - df_temp.cabg_pr)*df_temp.true_cate)
    return true_cb_asre

#### Run these functions on the large simulated dataframe ( `nsim` observations)

In [15]:
true_are = compute_true_are(large_df)

true_1_3_r_asre = compute_true_r_asre(large_df, 1/3)
true_2_3_r_asre = compute_true_r_asre(large_df, 2/3)

true_95_ci_asre = compute_true_ci_asre(large_df, .05)
true_55_ci_asre = compute_true_ci_asre(large_df, .45)

true_1_3_cb_asre = compute_true_cb_asre(large_df, 1/3)
true_2_3_cb_asre = compute_true_cb_asre(large_df, 2/3)